# Fase 1: Importar las dependencias

In [ ]:
!pip install sentencepiece
!pip install tf-models-nightly
!pip install tf-nightly
!pip install python-binance

# Fase 2: CREAR ORDER DE COMPRA-VENTA

In [ ]:
API_KEY = ""
API_SECRET= ""
TRADE_QUANTITY = 100

from binance.client import Client
from binance.enums import *

In [ ]:

client = Client(API_KEY, API_SECRET, tld='us')

def order(side, quantity, symbol,order_type=ORDER_TYPE_MARKET):
    try:
        print("sending order")
        order = client.create_order(symbol=symbol, side=side, type=order_type, quantity=quantity)
        print(order)
    except Exception as e:
        print("an exception occured - {}".format(e))
        return False

    return True

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_hub as hub

In [ ]:
tf.__version__

In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass
import tensorflow as tf

In [ ]:
from official.nlp.bert.tokenization import FullTokenizer
from official.nlp import optimization

In [ ]:
import numpy as np
import math
import re
import pandas as pd
from bs4 import BeautifulSoup
import random

from google.colab import drive

# Fase 2: Pre Procesado de Datos

## Carga de Ficheros

We import files from our personal Google drive.

In [ ]:
drive.mount("/content/drive")

In [ ]:
cols = ["sentiment", "id", "date", "query", "user", "text"]
data = pd.read_csv(
    "PONER AQUI EL DATASET DE SENTIMIENTOS",
    header=None,
    names=cols,
    engine="python",
    encoding="latin1"
)

In [ ]:
data.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

## Pre Procesado

### Limpieza

In [ ]:
def clean_tweet(tweet):
    tweet = BeautifulSoup(tweet, "lxml").get_text()
    # Removing the @
    tweet = re.sub(r"@[A-Za-z0-9]+", ' ', tweet)
    # Removing the URL links
    tweet = re.sub(r"https?://[A-Za-z0-9./]+", ' ', tweet)
    # Keeping only letters
    tweet = re.sub(r"[^a-zA-Z.!?']", ' ', tweet)
    # Removing additional whitespaces
    tweet = re.sub(r" +", ' ', tweet)
    return tweet

In [ ]:
data_clean = [clean_tweet(tweet) for tweet in data.text]

In [ ]:
data_labels = data.sentiment.values
data_labels[data_labels == 4] = 1

### Tokenización

We need to create a BERT layer to have access to meta data for the tokenizer (like vocab size).

In [ ]:
FullTokenizer = FullTokenizer
bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
                            trainable=False)
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = FullTokenizer(vocab_file, do_lower_case)

We only use the first sentence for BERT inputs so we add the CLS token at the beginning and the SEP token at the end of each sentence.

In [ ]:
def encode_sentence(sent):
    return ["[CLS]"] + tokenizer.tokenize(sent) + ["[SEP]"]

In [ ]:
data_inputs = [encode_sentence(sentence) for sentence in data_clean]

We need to create the 3 different inputs for each sentence.

In [ ]:
def get_ids(tokens):
    return tokenizer.convert_tokens_to_ids(tokens)
    
def get_mask(tokens):
    return np.char.not_equal(tokens, "[PAD]").astype(int)

def get_segments(tokens):
    seg_ids = []
    current_seg_id = 0
    for tok in tokens:
        seg_ids.append(current_seg_id)
        if tok == "[SEP]":
            current_seg_id = 1-current_seg_id # turns 1 into 0 and vice versa
    return seg_ids

### Creación del Dataset

We will create padded batches (so we pad sentences for each batch independently), this way we add the minimum of padding tokens possible. For that, we sort sentences by length, apply padded_batches and then shuffle.

In [ ]:
data_with_len = [[sent, data_labels[i], len(sent)]
                 for i, sent in enumerate(data_inputs)]
random.shuffle(data_with_len)
data_with_len.sort(key=lambda x: x[2])
sorted_all = [([get_ids(sent_lab[0]),
                get_mask(sent_lab[0]),
                get_segments(sent_lab[0])],
               sent_lab[1])
              for sent_lab in data_with_len if sent_lab[2] > 7]

In [ ]:
# A list is a type of iterator so it can be used as generator for a dataset
all_dataset = tf.data.Dataset.from_generator(lambda: sorted_all,
                                             output_types=(tf.int32, tf.int32))

In [ ]:
BATCH_SIZE = 32
all_batched = all_dataset.padded_batch(BATCH_SIZE,
                                       padded_shapes=((3, None), ()),
                                       padding_values=(0, 0))

In [ ]:
NB_BATCHES = len(sorted_all) // BATCH_SIZE
NB_BATCHES_TEST = NB_BATCHES // 10
NB_BATCHES_TRAIN = NB_BATCHES - NB_BATCHES_TEST
all_batched.shuffle(NB_BATCHES)
test_dataset = all_batched.take(NB_BATCHES_TEST)
train_dataset = all_batched.skip(NB_BATCHES_TEST)

# Fase 3: Construcción del modelo

In [ ]:
class BERTClassifier(tf.keras.Model):

    def __init__(self,
                 nb_classes=2,
                 dropout_rate=0.1,
                 name="bert_classifier"):
        super(BERTClassifier, self).__init__(name=name)

        self.dropout_rate = dropout_rate

        self.bert_layer = hub.KerasLayer(
            "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1",
            trainable=True)
        self.last_dense = layers.Dense(
            units=nb_classes,
            kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))
    
    def apply_bert(self, all_tokens):
        pooled_output, _ = self.bert_layer([all_tokens[:, 0, :],
                                            all_tokens[:, 1, :],
                                            all_tokens[:, 2, :]])
        
        return pooled_output
    
    def call(self, inputs, training):
        output = self.apply_bert(inputs)
        output = tf.nn.dropout(output, rate=self.dropout_rate)

        probs = self.last_dense(output)

        return probs

# Fase 4: Entrenamiento

In [ ]:
NB_CLASSES = 2

DROPOUT_RATE = 0.1

BATCH_SIZE = 32
NB_EPOCHS = 5
INIT_LR = 5e-5
WARMUP_STEPS = int(NB_BATCHES_TRAIN * 0.1)

In [ ]:
bert_classifier = BERTClassifier(NB_CLASSES, DROPOUT_RATE)

In [ ]:
NB_BATCHES_TRAIN

In [ ]:
train_dataset_light = train_dataset.take(500)
steps_per_epoch_light = 100
WARMUP_STEPS_LIGHT = int(500 * 0.1)

In [ ]:
optimizer_light = optimization.create_optimizer(
    init_lr=INIT_LR,
    num_train_steps=500,
    num_warmup_steps=WARMUP_STEPS_LIGHT)

In [ ]:
# very close but slightly better than standard categorical crossentropy loss
def classification_loss_fn(labels, logits):
    labels = tf.squeeze(labels)
    log_probs = tf.nn.log_softmax(logits, axis=-1)
    one_hot_labels = tf.one_hot(
        tf.cast(labels, dtype=tf.int32), depth=NB_CLASSES, dtype=tf.float32)
    per_example_loss = -tf.reduce_sum(
        tf.cast(one_hot_labels, dtype=tf.float32) * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return loss

In [ ]:
bert_classifier.compile(optimizer_light,
                        classification_loss_fn,
                        [tf.keras.metrics.SparseCategoricalAccuracy()])

In [ ]:
checkpoint_path = "/content/drive/My Drive/Model/BERT/"

ckpt = tf.train.Checkpoint(bert_classifier=bert_classifier)


ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=1)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

In [ ]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [ ]:
bert_classifier.fit(train_dataset_light,
                    steps_per_epoch=100,
                    epochs=NB_EPOCHS,
                    callbacks=[MyCustomCallback()])

# Stage 5: Evaluation

In [ ]:
bert_classifier.evaluate(test_dataset)

In [ ]:
def get_prediction(sentence):
    tokens = encode_sentence(sentence)

    input_ids = get_ids(tokens)
    input_mask = get_mask(tokens)
    segment_ids = get_segments(tokens)

    inputs = tf.stack(
        [tf.cast(input_ids, dtype=tf.int32),
         tf.cast(input_mask, dtype=tf.int32),
         tf.cast(segment_ids, dtype=tf.int32)],
         axis=0)
    inputs = tf.expand_dims(inputs, 0) # simulates a batch

    output = bert_classifier(inputs, training=False)

    sentiment = tf.argmax(tf.squeeze(output)).numpy()

    if sentiment == 0:
        print("Output of the model: {}\nPredicted sentiment: negative".format(
            output))
        return 0
    elif sentiment == 1:
        print("Output of the model: {}\nPredicted sentiment: positive".format(
            output))
        return True

In [ ]:
import tweepy

api_key = ""
api_secret_key = "
access_token = ""
access_token_secret = ""

# Create The Authenticate Object
authenticate = tweepy.OAuthHandler(api_key, api_secret_key)

# Set The Access Token & Access Token Secret
authenticate.set_access_token(access_token, access_token_secret)

# Create The API Object
api = tweepy.API(authenticate, wait_on_rate_limit = True)

In [ ]:
#Filters out mentions and RTs
def from_creator(status):
    if hasattr(status, 'retweeted_status'):
        return False
    elif status.in_reply_to_status_id != None:
        return False
    elif status.in_reply_to_screen_name != None:
        return False
    elif status.in_reply_to_user_id != None:
        return False
    else:
        return True

class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status):
        if from_creator(status):
            tweet = status.text.lower()
            value = get_prediction(tweet)
            if "doge" in tweet:
              if value:
                order(TRADE_QUANTITY, SIDE_BUY, "DOGEUSD",order_type=ORDER_TYPE_MARKET)
              else:
                order(TRADE_QUANTITY, SIDE_SELL, "DOGEUSD",order_type=ORDER_TYPE_MARKET)
            if "bitcoin" in tweet:
              if value:
                order(TRADE_QUANTITY, SIDE_BUY, "BTCUSD",order_type=ORDER_TYPE_MARKET)
              else:
                order(TRADE_QUANTITY, SIDE_SELL, "BTCUSD",order_type=ORDER_TYPE_MARKET)
            return True
        return True


    def on_error(self, status_code):
        if status_code == 420:
            print("Error 420")
            #returning False in on_error disconnects the stream
            return False
    
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener)   
myStream.filter(follow=['44196397'])             
